In [7]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : c:\
Directory name is : 


In [956]:
import os
import glob
import pandas

path = 'C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]
print(result)

['F-179.csv', 'F-319.csv', 'F-319_clean.csv', 'F-45.csv', 'G-1220.csv', 'G-1260_Done.csv', 'G-2147.csv', 'G-2866.csv', 'G-3549.csv', 'G-561.csv', 'G-580A.csv', 'G-852_NO_TIDE.csv', 'G-860.csv', 'PB-1680.csv', 'Well F-179_FINAL.csv']


# Read file

In [487]:
# Dont do number 4
# Next to do: g1260, 11 (NO TIDE)

In [488]:
start = '2007-10-01 01:00:00'
end = '2018-06-08 11:00:00'
time_range = pd.date_range(start=start, end=end,freq='H')

In [1247]:
file = result[5]
df = pd.read_csv(file)

#if 'Code' in df:
  # df = df[df.Code == "A"]
file

'G-1260_Done.csv'

In [1241]:
df.Code.describe()

count     97220
unique        2
top           A
freq      88731
Name: Code, dtype: object

In [1242]:
df = df[df.Code != "P"]

In [1243]:
df.Code.unique()

array(['A', nan], dtype=object)

In [1038]:
df[df.RainDateHour == '2018/04/09 11:00']

,TideDateHour,Date,Time,Tide_ft,WellDateHour,date,time,tz_cd,Well_ft,Code,Corrected,RainDateHour,Date.1,RAIN_FT
368972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018/04/09 11:00,4/9/18 11:00,0.0


# Aggregate Tide Water Elevation

In [ ]:
#%% Create a set format for which to read the string as a datetime variable

df['TideDateHour'] = df.loc[:, 'DateHour']
df['Tide_ft'] = df.loc[:, 'TideLevel']
tide = df.loc[:, ['TideDateHour', 'Tide_ft']]

tide = tide.loc[tide.TideDateHour != '1900/01/00 00:00']

time_format = "%m/%d/%Y %H:%M"

tide['TideDateHour'] = pd.to_datetime(tide.loc[:,'TideDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
tide = tide.set_index('TideDateHour')

#%% Select dates in the range required
data_tide = tide['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_tide = data_tide.resample('H').mean()

In [1244]:
# Aggregate Rain

#%% Create a set format for which to read the string as a datetime variable
rain = df.loc[:, ['RainDateHour', 'RAIN_FT']]

time_format = "%Y/%m/%d %H:%M"

rain['RainDateHour'] = pd.to_datetime(rain['RainDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
rain = rain.set_index('RainDateHour')

#%% Select dates in the range required
data_rain = rain['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_rain = data_rain.resample('H').mean()

# Aggregate Well

#%% Create a set format for which to read the string as a datetime variable
well = df.loc[:, ['WellDateHour', 'Well_ft']]

well = well.loc[well.WellDateHour != '1900/01/00 00:00']
well['WellDateHour'] = pd.to_datetime(well['WellDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
well = well.set_index('WellDateHour')

#%% Select dates in the range required
data_narrow = well['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_well = data_narrow.resample('H').mean()

data_well['Well_ft'] = data_well.Well_ft.interpolate()

C:\Users\Christopher\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.
C:\Users\Christopher\Anaconda3\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
C:\Users\Christopher\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the docume

TypeError: invalid type comparison

# Merge all of the data into one table and save

In [1227]:
file

'PB-1680.csv'

In [1228]:
len(time_range)

93683

In [1230]:
full = pd.DataFrame(index=pd.DatetimeIndex(time_range))

In [1231]:
full.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 93683 entries, 2007-10-01 01:00:00 to 2018-06-08 11:00:00
Freq: H
Empty DataFrame

In [1232]:
full['Well_ft'] = data_well['Well_ft']
full['Tide_ft'] = data_tide['Tide_ft']
full['Rain_ft'] = data_rain['RAIN_FT']
full.index.name = 'DateHour'

In [1233]:
full.Tide_ft.interpolate(inplace=True)
full.Rain_ft.interpolate(inplace=True)
full.Well_ft.interpolate(inplace=True)

# Fill in empty begininngs and ends
avg_tide = full.Tide_ft.mean()
full.Tide_ft.fillna(value=avg_tide, inplace=True)
full.Tide_ft.fillna(value=avg_tide, inplace=True)

full.Rain_ft.fillna(value=0, inplace=True)
full.Rain_ft.fillna(value=0, inplace=True)

avg_well = full.Well_ft.mean()
full.Well_ft.fillna(value=avg_well, inplace=True)
full.Well_ft.fillna(value=avg_well, inplace=True)


In [1234]:
assert not sum(full.Tide_ft.isna())
assert not sum(full.Rain_ft.isna())
assert not sum(full.Well_ft.isna())

In [1235]:
new_file = "final_wells/" + file[0:len(file)-4].replace("-", "") + "_ALL.csv"

In [1236]:
new_file

'final_wells/PB1680_ALL.csv'

In [1237]:
full.to_csv(new_file, index=True)

# Incorporate Predicted Well Water Elevation After SAS Prediction

In [1298]:
file = result[15]
well = file[:len(file)-4].replace("_", "").replace("-", "")
well_name = "Well " + well #file[:len(file)-9]
print(well_name)
print(well)

IndexError: list index out of range

In [1295]:
pred = pd.read_sas("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + str(well) + "_all_pred.sas7bdat")
full = pd.read_csv("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + str(well) + "_ALL.csv", index_col='DateHour')
pred.set_index(full.index, inplace=True)

full[well_name + " Well Level"] = full.Well_ft
full[well_name + " Tide Level"] = full.Tide_ft
full[well_name + " Rain Level"] = full.Rain_ft
full[well_name + " Predicted Well Level"] = pred['FORECAST']
full[well_name + " Lower 95% Confidence Interval"] = pred.L95
full[well_name + " Upper 95% Confidence Interval"] = pred.U95

full.head()

,Well_ft,Tide_ft,Rain_ft,Well PB1680 Well Level,Well PB1680 Tide Level,Well PB1680 Rain Level,Well PB1680 Predicted Well Level,Well PB1680 Lower 95% Confidence Interval,Well PB1680 Upper 95% Confidence Interval
DateHour,,,,,,,,,
2007-10-01 01:00:00,4.43,-0.098,0.000000,4.43,-0.098,0.000000,NaN,NaN,NaN
2007-10-01 02:00:00,4.44,0.000,0.001333,4.44,0.000,0.001333,NaN,NaN,NaN
2007-10-01 03:00:00,4.44,-0.273,0.000000,4.44,-0.273,0.000000,NaN,NaN,NaN
2007-10-01 04:00:00,4.44,-0.822,0.000000,4.44,-0.822,0.000000,NaN,NaN,NaN
2007-10-01 05:00:00,4.44,-1.475,0.000000,4.44,-1.475,0.000000,4.434899,4.410783,4.459015


In [1296]:
#full = full.iloc[:, 3:]
full.to_csv("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + str(well) + "_FINAL.csv", index=True)

# Aggregate all data

In [1299]:
import os
import glob
import pandas

path = 'C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]

term = result[1]
print(term)
print(term[(len(term)-9):len(term)])
result = [i for i in result if i[(len(i)-9):len(i)] == "FINAL.csv"]
print(result)

F179_ALL.csv
9_ALL.csv
['F179_FINAL.csv', 'F319_FINAL.csv', 'F45_FINAL.csv', 'G1220_FINAL.csv', 'G1260_FINAL.csv', 'G2147_FINAL.csv', 'G2866_FINAL.csv', 'G3549_FINAL.csv', 'G561_FINAL.csv', 'G580A_FINAL.csv', 'G860_FINAL.csv', 'PB1680_FINAL.csv']


In [1300]:
df = pd.DataFrame(index=time_range)

In [1339]:
well = pd.read_csv(path + result[1])

In [1340]:
well.head(10)

,DateHour,Well_ft,Tide_ft,Rain_ft,Well F319 Well Level,Well F319 Tide Level,Well F319 Rain Level,Well F319 Predicted Well Level,Well F319 Lower 95% Confidence Interval,Well F319 Upper 95% Confidence Interval
0,2007-10-01 01:00:00,3.05,0.965,0.005229,3.05,0.965,0.005229,NaN,NaN,NaN
1,2007-10-01 02:00:00,3.05,0.358,0.004708,3.05,0.358,0.004708,NaN,NaN,NaN
2,2007-10-01 03:00:00,3.06,-0.144,0.005083,3.06,-0.144,0.005083,NaN,NaN,NaN
3,2007-10-01 04:00:00,3.06,-0.623,0.005771,3.06,-0.623,0.005771,NaN,NaN,NaN
4,2007-10-01 05:00:00,3.09,-0.994,0.001708,3.09,-0.994,0.001708,3.140221,3.112755,3.167688
5,2007-10-01 06:00:00,3.16,-1.168,0.005583,3.16,-1.168,0.005583,3.165017,3.137550,3.192483
6,2007-10-01 07:00:00,3.16,-0.846,0.001938,3.16,-0.846,0.001938,3.202239,3.174773,3.229705
7,2007-10-01 08:00:00,3.24,-0.509,0.001750,3.24,-0.509,0.001750,3.233813,3.206346,3.261279
8,2007-10-01 09:00:00,3.51,0.125,0.000375,3.51,0.125,0.000375,3.272549,3.245083,3.300016
9,2007-10-01 10:00:00,3.55,0.584,0.006042,3.55,0.584,0.006042,3.537308,3.509842,3.564774


In [1302]:
df['Well Level'] = well.iloc[:, 0]
df['Tide Level'] = well.iloc[:, 1]
df['Rain Level'] = well.iloc[:, 2]
df['Predicted Well Level'] = well.iloc[:,3]
df['Well Name'] = result[0]

In [1303]:
df = pd.DataFrame(new_df)

In [1304]:
df.head()

,DateHour,Predicted Well Level,Rain Level,Tide Level,Well Level,Well Name
0,2007-10-01 01:00:00,3.1775,0.001771,0.965,3.1775,F179
1,2007-10-01 02:00:00,3.1900,0.002146,0.358,3.1900,F179
2,2007-10-01 03:00:00,3.2075,0.005104,-0.144,3.2075,F179
3,2007-10-01 04:00:00,3.2275,0.004792,-0.623,3.2275,F179
4,2007-10-01 05:00:00,3.2550,0.002188,-0.994,3.2550,F179


In [1361]:
len(df)/12

93683.0

In [1360]:
i=1
well = pd.read_csv(path + result[i])

new_df = {
    'DateHour': well.iloc[:, 0],
    'Well Level': well.iloc[:, 1],
    'Tide Level': well.iloc[:, 2],
    'Rain Level': well.iloc[:, 3],
    'Predicted Well Level': well.iloc[:,7],
    'Well Name': pd.Series([result[i][:len(result[i])-10] for name in range(len(well))])
}
#print(well.head())
#df = pd.DataFrame(new_df)

df = df.append(pd.DataFrame(new_df))
print(df.tail())
print(len(df))

IndexError: list index out of range

In [1318]:
df['Well Name'].unique()

array(['F179', 'F319', 'F45', 'G1220', 'G1260', 'G2147', 'G2866', 'G3549',
       'G561', 'G580A', 'G860', 'PB1680'], dtype=object)

In [1362]:
df.to_csv(path + "Combined.csv", index=False)

In [1363]:
df.tail(168)

,DateHour,Predicted Well Level,Rain Level,Tide Level,Well Level,Well Name
93515,2018-06-01 12:00:00,2.248987,0.0,-1.937,2.25,PB1680
93516,2018-06-01 13:00:00,2.247563,0.0,-1.937,2.25,PB1680
93517,2018-06-01 14:00:00,2.245858,0.0,-1.937,2.25,PB1680
93518,2018-06-01 15:00:00,2.243967,0.0,-1.937,2.25,PB1680
93519,2018-06-01 16:00:00,2.241922,0.0,-1.937,2.25,PB1680
93520,2018-06-01 17:00:00,2.239782,0.0,-1.937,2.25,PB1680
93521,2018-06-01 18:00:00,2.237588,0.0,-1.937,2.25,PB1680
93522,2018-06-01 19:00:00,2.235364,0.0,-1.937,2.25,PB1680
93523,2018-06-01 20:00:00,2.233122,0.0,-1.937,2.25,PB1680
93524,2018-06-01 21:00:00,2.230873,0.0,-1.937,2.25,PB1680


In [1328]:
test = df.set_index(pd.DatetimeIndex(df.DateHour))

In [1336]:
#Get MAPE for each well prediction
# Mean(yi - yhat / yi)
test = test['2018-06-01 12:00:00':'2018-06-08 11:00:00']
test['res'] = abs(test['Well Level'] - test['Predicted Well Level'])/abs(test['Well Level'])

test.groupby('Well Name')['res'].mean()

Well Name
F179      0.000000
F319      0.000000
F45       0.000000
G1220     0.000000
G1260     0.043546
G2147     0.000000
G2866     0.000000
G3549     0.000000
G561      0.000000
G580A     0.000000
G860      0.000000
PB1680    0.000000
Name: res, dtype: float64

# Testing for missing rain data 10/24

In [922]:
df.set_index(pd.DatetimeIndex(df['DateHour']), inplace=True)

In [953]:
sub1 = df.loc['11/04/2010 11:00:00':]
sub1 = sub1[sub1['Well Name'] == "G3549"]

In [933]:
sub = pd.read_csv(path + "../G-3549.csv")

C:\Users\Christopher\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,4,5,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [935]:
sub.set_index(pd.DatetimeIndex(sub.RainDateHour), inplace=True)

In [946]:
sub = sub.loc['11/04/2010 11:00:00':'11/04/2017']